## Baseline code
- 자전거 수요 예측
- target: count컬럼

- 제출 파일 : datetime, count

In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv("/kaggle/input/bike-sharing-demand/train.csv")
test = pd.read_csv("/kaggle/input/bike-sharing-demand/test.csv")

## EDA

In [ ]:
# 데이터 크기
train.shape, test.shape

In [ ]:
# 데이터 샘플
train.head(2)

In [ ]:
# 데이터 샘플
test.head(2)

- casual, registered의 합이 count인 것을 확인할 수 있음 (test에는 없으므로 제거)

In [ ]:
# 결측값 확인
train.isnull().sum()

In [ ]:
# 결측값 확인
test.isnull().sum()

In [ ]:
train['count'].hist()

## 전처리

In [ ]:
# datetime

train['datetime'] = pd.to_datetime(train['datetime'])
test['datetime'] = pd.to_datetime(test['datetime'])

train['year'] = train['datetime'].dt.year
train['month'] = train['datetime'].dt.month
train['day'] = train['datetime'].dt.day

test['year'] = test['datetime'].dt.year
test['month'] = test['datetime'].dt.month
test['day'] = test['datetime'].dt.day

train = train.drop('datetime', axis=1)
test = test.drop('datetime', axis=1)

In [ ]:
# test에는 없는 컬럼 삭제
train = train.drop(['casual', 'registered'], axis=1)
train.head(1)

In [ ]:
# target 별도 저장
target = train.pop('count')
target

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=2023)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

## 선형회귀 모델

In [ ]:
# 평가
from sklearn.metrics import r2_score

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
pred = lr.predict(X_val)
r2_score(y_val, pred)

## 랜덤포레스트

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
pred = rf.predict(X_val)
r2_score(y_val, pred)

## XGBoost 모델

In [ ]:
from xgboost import XGBRegressor
xgb = XGBRegressor()
xgb.fit(X_train, y_train)
pred = xgb.predict(X_val)
r2_score(y_val, pred)

In [ ]:
# 하이퍼파라미터
from xgboost import XGBRegressor
xgb = XGBRegressor(n_estimators=500, learning_rate=0.01, max_depth=9)
xgb.fit(X_train, y_train)
pred = xgb.predict(X_val)
r2_score(y_val, pred)

## test 데이터 예측

In [ ]:
# 예측
pred = xgb.predict(test)
pred

In [ ]:
# csv 파일 생성
test = pd.read_csv("/kaggle/input/bike-sharing-demand/test.csv")

submit = pd.DataFrame({
    'datetime': test['datetime'],
    'count': pred
})
submit.to_csv("submission.csv", index=False)

In [ ]:
# 점검
print(pd.read_csv("submission.csv"))
print(submit.shape, test.shape)